In [66]:
'''1. Remove foreign records to a 'foreign dataframe' and delete them from this df. - applicable for  address
2.Find whether all phone numbers are empty, if empty drop those rows (8708)
3.Column Phone matches the Phone number in preferred phone number
Transfer records with non-null values in Phone to a new dataframe with columns EID and phone
Find the count of records (124000)
4. Null values in Phone: (36000 records) Create a new df:Check the content in preferred phone 
and grasp the preferred phone and add it to df with columns eid and phone
'''
import pandas as pd

df_txexEmail = pd.read_csv('TexasExes.csv')
print("original count ",len(df_txexEmail))
df_txexEmail = df_txexEmail[['EID','Email','Personal Email','Alternate Email','Work Email']]
print(df_txexEmail.head(3))

df_Email = df_txexEmail.dropna(how='all', subset=['Email','Personal Email','Alternate Email','Work Email'])
print("Count records after dropping records with no emails : ",len(df_Email))

df_Email_nb = df_Email.dropna(subset=['Email'])
print("Count records with values in column email : ",len(df_Email_nb))

del df_Email_nb, df_txexEmail


original count  2200
        EID                  Email         Personal Email Alternate Email  \
0    to3269  taylor.orth@gmail.com  taylor.orth@gmail.com             NaN   
1  0563numt   fishbaughm@gmail.com   fishbaughm@gmail.com             NaN   
2     ksn99    ksn2006@hotmail.com    ksn2006@hotmail.com             NaN   

  Work Email  
0        NaN  
1        NaN  
2        NaN  
Count records after dropping records with no emails :  1271
Count records with values in column email :  1271


In [67]:
df_Email = df_Email[['EID','Email']]
df_Email['EID'] = df_Email['EID'].str.upper()

In [68]:
raw_eid = df_Email['EID'].dropna()
eid = raw_eid.tolist()
#print(len(eid))
# How many elements each list should have 
n = 1000
pid = {}
for i in range((len(eid) + n - 1) // n ):
    #globals()["eid" + str(i)] = eid[i * n:(i + 1) * n]
    pid[i] = eid[i * n:(i + 1) * n]

In [70]:
import cx_Oracle
connection = cx_Oracle.connect("xxxx/Zyyy@ea.y.z.com:1525/z.x.y.com")
cursor = connection.cursor()
df_edb = pd.DataFrame(columns = ['EID','Email'])
for i in range(2):
    cursor.execute("select UT_EID, vi_email_addr from vi_vips_main m, vi_vips_email_info e where m.ALUMNI_ID = e.ALUMNI_ID AND m.ut_eid IN %s" % str(tuple(pid[i])).replace(',)',')'))
    results = cursor.fetchall()
    df1 = pd.DataFrame(results, columns = ['EID','Email'])
    df_edb = df_edb.append(df1,ignore_index=True)
cursor.close()
connection.close()

In [71]:
print(df_edb.head(3))
print(len(df_edb))
df_ecsv = df_Email
print(df_ecsv.head(2))
print(len(df_ecsv))
del df_Email

        EID                  Email
0   DMD3369    18duncand@gmail.com
1    TN9386  20nwafort@smtexas.org
2  0669DWRG           2jgs@msn.com
1525
        EID                  Email
0    TO3269  taylor.orth@gmail.com
1  0563NUMT   fishbaughm@gmail.com
1271


In [72]:
df_email = pd.DataFrame()
for i in range(len(df_ecsv)):
    id_csv = df_ecsv.iloc[i,0]
    email_csv = df_ecsv.iloc[i,1]
    email_db = (df_edb.loc[df_edb['EID'] == id_csv]).iloc[:,1]
    email_db = {*email_db}
    email_csv = {email_csv}
    diff = (email_csv - email_db)
    if len(diff) > 0:
        #df_email = df_email.append({'EID':id_csv, 'Email':diff}, ignore_index=True)
        df_email = df_email.append({'EID':id_csv, 'Email_csv':df_ecsv.iloc[i,1], 'Email_db':email_db}, ignore_index=True)

In [73]:
print(df_email.head(4))
print(len(df_email))
#df_email = df_email.drop_duplicates()
print(len(df_email))

       EID                 Email_csv Email_db
0  KTBOOSE     katysombito@gmail.com       {}
1    WK589    keeshond6919@yahoo.com       {}
2  KHW2314  kendra.h.allen@gmail.com       {}
3   RCP524       clpresent@gmail.com       {}
81
81


In [74]:
df_email.to_csv(r'C:\Users\le5752\Documents\DB_Python\EmailsDiff.csv', index = False)